<a href="https://colab.research.google.com/github/Rahulraj31/Diabetic-Retinopathy/blob/main/Diabetic_Retinopathy_224x224_(2019_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
import PIL.Image as Image
import cv2
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline 
from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [4]:
df= pd.read_csv("/content/train.csv")
df.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


0 - No_DR
1 - Mild
2 - Moderate
3 - Severe
4 - Proliferate_DR

In [8]:
from pathlib import Path 
import os
train_data_dir = Path('/content/colored_images')

In [9]:
train_images_dict = {
    'Mild': list(train_data_dir.glob('Mild/*')) ,   
    'Moderate': list(train_data_dir.glob('Moderate/*')),
    'No_DR': list(train_data_dir.glob('No_DR/*')),
    'Proliferate_DR': list(train_data_dir .glob('Proliferate_DR/*')),
    'Severe': list(train_data_dir.glob('Severe/*')),
}

labels_dict = {
     'Mild': 'mild' ,   
    'Moderate':'moderate',
    'No_DR': 'no_dr',
    'Proliferate_DR':'proliferate_dr',
    'Severe': 'sever',
}

In [13]:
img= cv2.imread(str(train_images_dict['Mild'][0]))
img.shape

(224, 224, 3)